#  대화 이력 관리를 위한 메모리 구현(Chat History)

### **학습 목표:**  대화 이력 관리를 위한 메모리 컴포넌트 구현 방법을 실습한다

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [4]:
import os
from glob import glob

from pprint import pprint
import json

`(3) LLM 설정`

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.3,
    top_p=0.9, 
)

# **채팅 히스토리 관리**

* 채팅봇의 핵심 기능은 이전 대화 내용을 문맥으로 활용하는 것으로, 이를 통해 자연스러운 대화 흐름을 유지할 수 있습니다.

* 가장 기본적인 방식은 이전 메시지들을 모델의 프롬프트에 직접 포함시키는 것이지만, 오래된 메시지는 적절히 제거하여 모델이 처리해야 할 정보량을 조절할 수 있습니다.

* 장시간 진행되는 대화의 경우, 단순히 이전 메시지를 저장하는 것을 넘어 대화 내용을 요약하여 저장하는 등의 고급 메모리 관리 기법을 활용할 수 있습니다.

### 1. **메시지 전달 방식(Message Passing)**

* 메시지 전달 방식은 LangChain에서 가장 기본적인 메모리 구현 방법으로, 이전 대화 기록(chat history)을 체인에 직접 전달하여 문맥을 유지하는 방식입니다.

* 이 방식은 SystemMessage(시스템 지시사항), HumanMessage(사용자 입력), AIMessage(AI 응답) 등 다양한 유형의 메시지를 ChatPromptTemplate을 통해 구조화하며, MessagesPlaceholder를 사용하여 이전 대화 내용을 포함시킵니다.

* 챗봇의 기본적인 메모리 시스템을 구현하는데 사용되며, 이를 통해 AI는 이전 대화 맥락을 이해하고 그에 맞는 적절한 응답을 생성할 수 있습니다.


In [6]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# ChatPromptTemplate를 사용하여 챗봇의 초기 메시지를 정의
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant."),
    MessagesPlaceholder(variable_name="messages"),    # 메시지 목록을 동적으로 삽입하는 부분
])

# ChatPromptTemplate에 삽입할 메시지 목록을 정의
messages = [
        HumanMessage(content="안녕하세요. 제 이름은 홍길동입니다."),
        AIMessage(content="안녕하세요! 어떻게 도와드릴까요?"),
]    

# ChatPromptTemplate에 삽입할 메시지 목록을 업데이트하고 출력 
pprint(prompt.format(messages=messages))

('System: You are a helpful assistant.\n'
 'Human: 안녕하세요. 제 이름은 홍길동입니다.\n'
 'AI: 안녕하세요! 어떻게 도와드릴까요?')


In [7]:
# 대화형 체인을 정의 (prompt -> llm)
chain = prompt | llm

# 기본적인 메시지 전달: 이전 메시지 목록에 새로운 메시지를 추가해서 전달
response = chain.invoke({
    "messages": messages + [HumanMessage(content="제 이름을 기억하나요?")] # 이전 메시지를 기억하는지 확인하는 질문 메시지 추가
})

pprint(response)

AIMessage(content='네, 제 이름은 홍길동이라고 기억하고 있습니다. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 53, 'total_tokens': 74, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_a150906e27', 'id': 'chatcmpl-CEvbnGpB10gaSTR1q9fHgiV9Xu6cF', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--4467b991-4776-416e-8efd-502d7a3e929f-0', usage_metadata={'input_tokens': 53, 'output_tokens': 21, 'total_tokens': 74, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})


### 2. **RunnableWithMessageHistory**

* RunnableWithMessageHistory는 LangChain에서 대화 기록을 관리하는 고급 기능으로, 체인의 실행 과정에서 메시지 기록을 자동으로 저장하고 검색할 수 있게 해주는 래퍼(wrapper) 클래스입니다.

* 이 기능은 대화 세션별로 독립적인 기록을 유지할 수 있게 해주며, ConfigurableField를 통해 메모리 구성을 유연하게 조정할 수 있습니다. 특히 여러 사용자와 동시에 대화할 때 각 세션의 컨텍스트를 분리하여 관리하는 데 매우 유용합니다.

* 주요 장점은 대화 기록 관리의 자동화와 일관성 있는 메시지 처리이지만, 메모리 저장소 설정과 관리에 추가적인 구성이 필요하다는 점을 고려해야 합니다. 또한 Redis나 다른 외부 저장소와 통합하여 영구적인 대화 기록 보관도 가능합니다.

* 구현 시에는 get_session_history 콜백을 통해 세션ID별로 메시지 기록을 관리하며, 이를 통해 각 대화의 컨텍스트를 정확하게 유지할 수 있습니다.


`(1) 메모리 기반 로컬 저장소 활용`

* `InMemoryHistory` 클래스는 대화 이력의 기본 구조를 제공하며, BaseChatMessageHistory와 BaseModel을 상속받아 메시지를 메모리에서 효율적으로 관리합니다. 특히 messages 리스트를 통해 BaseMessage 객체들을 순차적으로 저장하고, add_messages와 clear 메서드로 히스토리를 유연하게 관리할 수 있습니다.

* 시스템의 핵심인 `store` 변수는 전역 딕셔너리로 구현되어 세션별 대화 이력을 구분하여 저장합니다. session_id를 키로 사용하여 각 세션의 InMemoryHistory 객체에 빠르게 접근할 수 있으며, 이를 통해 다중 사용자 환경에서도 효율적인 대화 관리가 가능합니다.

* `get_session_history` 함수는 세션 관리의 진입점 역할을 하며, 존재하지 않는 세션에 대해 자동으로 새로운 InMemoryHistory 객체를 생성하는 팩토리 패턴을 구현합니다. 이러한 구조를 통해 세션의 생명주기를 자동으로 관리하고 메모리 효율성을 높일 수 있습니다.

In [8]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage
from pydantic import BaseModel, Field
from typing import List

# 메모리 기반 히스토리 구현
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    
    def add_messages(self, messages: List[BaseMessage]) -> None:
        self.messages.extend(messages)
    
    def clear(self) -> None:
        self.messages = []

# 세션 저장소
store = {}

# 세션 ID로 히스토리 가져오기
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

# 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 여행 가이드입니다. 관광객에게 유용한 정보를 제공하세요."),
    MessagesPlaceholder(variable_name="history"), 
    ("human", "{input}")
])

# 프롬프트와 llm을 연결하여 체인 생성
chain = prompt | llm

# 히스토리 관리 추가  
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 지정된 세션 ID(tourist_1)를 사용하여체인 실행
response = chain_with_history.invoke(
    {
        "input": "서울에서 가볼만한 곳을 추천해주세요."
    },
    config={"configurable": {"session_id": "tourist_1"}}
)


print(f"여행 가이드 답변:\n{response.content}")

여행 가이드 답변:
서울에는 볼거리와 즐길 거리가 정말 많습니다! 몇 가지 추천드릴게요.

1. 경복궁 – 조선 시대의 대표 궁궐로, 전통 건축과 왕실 문화를 체험할 수 있어요. 매일 오전 10시와 오후 2시에 수문장 교대식도 볼 수 있습니다.

2. 북촌 한옥마을 – 전통 한옥이 잘 보존된 마을로, 골목길 산책하며 한국의 옛 정취를 느껴보세요.

3. 명동 – 쇼핑과 길거리 음식이 풍부한 번화가로, 패션과 뷰티 제품을 구경하기 좋습니다.

4. 남산서울타워 – 서울의 전경을 한눈에 볼 수 있는 전망대로, 특히 야경이 아름답습니다. 케이블카를 타고 올라가는 것도 추천해요.

5. 홍대 – 젊음의 거리로, 다양한 카페, 공연장, 예술 공간이 많아 활기찬 분위기를 즐길 수 있습니다.

6. 동대문 디자인 플라자(DDP) – 독특한 건축물과 함께 패션 마켓, 전시회가 열리는 곳입니다.

서울 방문 시 교통은 지하철이 편리하니, 교통카드를 준비하시면 좋습니다. 더 궁금한 점 있으면 말씀해 주세요!


In [10]:
# 대화 히스토리 출력 
history = get_session_history("tourist_1")

pprint(history.messages)

[HumanMessage(content='서울에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='서울에는 볼거리와 즐길 거리가 정말 많습니다! 몇 가지 추천드릴게요.\n\n1. 경복궁 – 조선 시대의 대표 궁궐로, 전통 건축과 왕실 문화를 체험할 수 있어요. 매일 오전 10시와 오후 2시에 수문장 교대식도 볼 수 있습니다.\n\n2. 북촌 한옥마을 – 전통 한옥이 잘 보존된 마을로, 골목길 산책하며 한국의 옛 정취를 느껴보세요.\n\n3. 명동 – 쇼핑과 길거리 음식이 풍부한 번화가로, 패션과 뷰티 제품을 구경하기 좋습니다.\n\n4. 남산서울타워 – 서울의 전경을 한눈에 볼 수 있는 전망대로, 특히 야경이 아름답습니다. 케이블카를 타고 올라가는 것도 추천해요.\n\n5. 홍대 – 젊음의 거리로, 다양한 카페, 공연장, 예술 공간이 많아 활기찬 분위기를 즐길 수 있습니다.\n\n6. 동대문 디자인 플라자(DDP) – 독특한 건축물과 함께 패션 마켓, 전시회가 열리는 곳입니다.\n\n서울 방문 시 교통은 지하철이 편리하니, 교통카드를 준비하시면 좋습니다. 더 궁금한 점 있으면 말씀해 주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 302, 'prompt_tokens': 40, 'total_tokens': 342, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-

In [11]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_history.invoke(
    {
        "input": "이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?"
    },
    config={"configurable": {"session_id": "tourist_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

여행 가이드 답변:
이전에 추천드린 장소 중에서 가장 인기 있는 곳은 경복궁과 명동입니다.

- **경복궁**은 한국의 역사와 문화를 대표하는 궁궐로, 외국인 관광객뿐만 아니라 국내 방문객에게도 매우 인기가 많습니다. 특히 수문장 교대식과 아름다운 전통 건축물이 많은 관심을 받고 있어요.

- **명동**은 서울의 대표적인 쇼핑 거리로, 패션과 뷰티 제품을 찾는 관광객들에게 매우 인기 있는 장소입니다. 다양한 길거리 음식과 활기찬 분위기 덕분에 많은 사람들이 찾습니다.

두 곳 모두 서울 여행에서 빼놓을 수 없는 명소로 손꼽히니, 일정에 맞춰 방문해 보시면 좋을 것 같아요!


In [12]:
# 대화 히스토리 출력
history = get_session_history("tourist_1")

pprint(history.messages)

[HumanMessage(content='서울에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='서울에는 볼거리와 즐길 거리가 정말 많습니다! 몇 가지 추천드릴게요.\n\n1. 경복궁 – 조선 시대의 대표 궁궐로, 전통 건축과 왕실 문화를 체험할 수 있어요. 매일 오전 10시와 오후 2시에 수문장 교대식도 볼 수 있습니다.\n\n2. 북촌 한옥마을 – 전통 한옥이 잘 보존된 마을로, 골목길 산책하며 한국의 옛 정취를 느껴보세요.\n\n3. 명동 – 쇼핑과 길거리 음식이 풍부한 번화가로, 패션과 뷰티 제품을 구경하기 좋습니다.\n\n4. 남산서울타워 – 서울의 전경을 한눈에 볼 수 있는 전망대로, 특히 야경이 아름답습니다. 케이블카를 타고 올라가는 것도 추천해요.\n\n5. 홍대 – 젊음의 거리로, 다양한 카페, 공연장, 예술 공간이 많아 활기찬 분위기를 즐길 수 있습니다.\n\n6. 동대문 디자인 플라자(DDP) – 독특한 건축물과 함께 패션 마켓, 전시회가 열리는 곳입니다.\n\n서울 방문 시 교통은 지하철이 편리하니, 교통카드를 준비하시면 좋습니다. 더 궁금한 점 있으면 말씀해 주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 302, 'prompt_tokens': 40, 'total_tokens': 342, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-

`(2) SQLite 데이터베이스 영구 저장소 활용`

* SQLite 통합 구현을 위해서는 먼저 메시지를 저장할 데이터베이스 테이블 구조를 정의하고, BaseChatMessageHistory를 상속받아 메시지 저장/조회 로직을 구현해야 합니다. 이때 세션 ID를 기준으로 대화 내용을 구분하여 관리합니다.

* 시스템 구현 시에는 메시지의 타입(Human/AI), 내용, 메타데이터, 타임스탬프 등의 정보를 체계적으로 저장하고, 필요할 때 효율적으로 검색하고 활용할 수 있도록 적절한 인덱싱 전략을 수립하는 것이 중요합니다.

In [13]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
import sqlite3
from typing import List
import json

class SQLiteChatMessageHistory(BaseChatMessageHistory):
    """ 
    SQLite 데이터베이스를 사용하여 챗봇 대화 히스토리를 저장하는 클래스

    Attributes:
        session_id (str): 세션 ID
        db_path (str): SQLite 데이터베이스 파일 경로

    """
    def __init__(self, session_id: str, db_path: str = "chat_history.db"):
        self.session_id = session_id
        self.db_path = db_path
        self._create_tables()
    
    def _create_tables(self):
        """데이터베이스 테이블 생성"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # 메시지 테이블 생성
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS messages (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                session_id TEXT,
                message_type TEXT,
                content TEXT,
                metadata TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        
        conn.commit()
        conn.close()
    
    def add_message(self, message: BaseMessage) -> None:
        """단일 메시지 추가"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            INSERT INTO messages (session_id, message_type, content, metadata)
            VALUES (?, ?, ?, ?)
        """, (
            self.session_id,
            message.__class__.__name__,
            message.content,
            json.dumps(message.additional_kwargs)
        ))
        
        conn.commit()
        conn.close()
    
    def add_messages(self, messages: List[BaseMessage]) -> None:
        """여러 메시지 추가"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        for message in messages:
            cursor.execute("""
                INSERT INTO messages (session_id, message_type, content, metadata)
                VALUES (?, ?, ?, ?)
            """, (
                self.session_id,
                message.__class__.__name__,
                message.content,
                json.dumps(message.additional_kwargs)
            ))
        
        conn.commit()
        conn.close()
    
    def clear(self) -> None:
        """세션의 모든 메시지 삭제"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            DELETE FROM messages WHERE session_id = ?
        """, (self.session_id,))
        
        conn.commit()
        conn.close()
    
    @property
    def messages(self) -> List[BaseMessage]:
        """저장된 메시지 조회"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            SELECT message_type, content, metadata
            FROM messages 
            WHERE session_id = ?
            ORDER BY created_at
        """, (self.session_id,))
        
        messages = []
        for row in cursor.fetchall():
            message_type, content, metadata = row
            if message_type == "HumanMessage":
                message = HumanMessage(content=content)
            else:
                message = AIMessage(content=content)
            
            if metadata:
                message.additional_kwargs = json.loads(metadata)
            
            messages.append(message)
        
        conn.close()
        return messages
    

# 세션 ID로 히스토리 가져오기
def get_chat_history(session_id: str) -> BaseChatMessageHistory:
    return SQLiteChatMessageHistory(session_id=session_id)

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

# 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 여행 가이드입니다. 관광객에게 유용한 정보를 제공하세요."),
    MessagesPlaceholder(variable_name="history"), 
    ("human", "{input}")
])

# 프롬프트와 llm을 연결하여 체인 생성
chain = prompt | llm

# 히스토리 관리 추가  
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 지정된 세션 ID(tourist_1)를 사용하여체인 실행
response = chain_with_history.invoke(
    {
        "input": "수원에서 가볼만한 곳을 추천해주세요."
    },
    config={"configurable": {"session_id": "tourist_1"}}
)


print(f"여행 가이드 답변:\n{response.content}")

여행 가이드 답변:
수원은 역사와 문화가 풍부한 도시로, 다양한 볼거리와 즐길 거리가 많습니다. 수원에서 가볼 만한 주요 명소를 추천해드릴게요.

1. **수원 화성**  
   - 조선 정조 시대에 건설된 성곽으로, 유네스코 세계문화유산에 등재되어 있습니다.  
   - 화성행궁, 장안문, 팔달문 등 다양한 건축물을 둘러볼 수 있고, 성곽을 따라 산책하기 좋습니다.  
   - 야간에는 조명으로 아름답게 빛나는 화성도 감상할 수 있습니다.

2. **화성행궁**  
   - 조선시대 임금이 수원에 머물 때 사용하던 임시 궁궐입니다.  
   - 전통 건축과 다양한 문화행사가 열려 역사 체험에 좋습니다.

3. **팔달문 시장**  
   - 전통시장으로 다양한 먹거리와 쇼핑을 즐길 수 있습니다.  
   - 수원 갈비, 떡갈비 등 지역 특산 음식도 맛볼 수 있어요.

4. **수원박물관**  
   - 수원의 역사와 문화를 한눈에 볼 수 있는 곳입니다.  
   - 가족 단위 방문객에게 추천합니다.

5. **광교 호수공원**  
   - 자연과 함께 산책하거나 자전거 타기 좋은 공원입니다.  
   - 카페와 레스토랑도 많아 휴식 공간으로 적합합니다.

6. **경기도박물관**  
   - 경기도 지역의 역사와 문화를 소개하는 박물관입니다.  
   - 다양한 전시와 체험 프로그램이 마련되어 있습니다.

이 외에도 수원에는 다양한 카페, 맛집, 문화 공간이 많으니 일정에 맞게 방문해 보세요. 도움이 필요하시면 언제든지 말씀해 주세요!


In [15]:
# 대화 히스토리 출력
history = get_chat_history("tourist_1")

pprint(history.messages)

[HumanMessage(content='수원에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='수원은 역사와 문화가 풍부한 도시로, 다양한 볼거리와 즐길 거리가 많습니다. 수원에서 가볼 만한 주요 명소를 추천해드릴게요.\n\n1. **수원 화성**  \n   - 조선 정조 시대에 건설된 성곽으로, 유네스코 세계문화유산에 등재되어 있습니다.  \n   - 화성행궁, 장안문, 팔달문 등 다양한 건축물을 둘러볼 수 있고, 성곽을 따라 산책하기 좋습니다.  \n   - 야간에는 조명으로 아름답게 빛나는 화성도 감상할 수 있습니다.\n\n2. **화성행궁**  \n   - 조선시대 임금이 수원에 머물 때 사용하던 임시 궁궐입니다.  \n   - 전통 건축과 다양한 문화행사가 열려 역사 체험에 좋습니다.\n\n3. **팔달문 시장**  \n   - 전통시장으로 다양한 먹거리와 쇼핑을 즐길 수 있습니다.  \n   - 수원 갈비, 떡갈비 등 지역 특산 음식도 맛볼 수 있어요.\n\n4. **수원박물관**  \n   - 수원의 역사와 문화를 한눈에 볼 수 있는 곳입니다.  \n   - 가족 단위 방문객에게 추천합니다.\n\n5. **광교 호수공원**  \n   - 자연과 함께 산책하거나 자전거 타기 좋은 공원입니다.  \n   - 카페와 레스토랑도 많아 휴식 공간으로 적합합니다.\n\n6. **경기도박물관**  \n   - 경기도 지역의 역사와 문화를 소개하는 박물관입니다.  \n   - 다양한 전시와 체험 프로그램이 마련되어 있습니다.\n\n이 외에도 수원에는 다양한 카페, 맛집, 문화 공간이 많으니 일정에 맞게 방문해 보세요. 도움이 필요하시면 언제든지 말씀해 주세요!', additional_kwargs={'refusal': None}, response_metadata={})]


In [16]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_history.invoke(
    {
        "input": "이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?"
    },
    config={"configurable": {"session_id": "tourist_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

여행 가이드 답변:
이전에 추천해드린 장소 중에서 가장 인기 있는 곳은 **수원 화성**입니다.

수원 화성은 조선 정조 시대의 대표적인 성곽으로, 역사적 가치가 뛰어나고 아름다운 건축물과 성곽 산책로가 잘 조성되어 있어 관광객들에게 매우 인기가 많습니다. 특히 유네스코 세계문화유산으로 지정되어 있어 국내외 방문객이 많이 찾는 명소입니다.

또한, 화성행궁과 팔달문 시장도 화성 방문과 함께 많이 찾는 장소로, 수원 화성 관광의 중심지 역할을 하고 있습니다. 야간 조명과 다양한 문화행사도 수원 화성의 매력을 더해주고 있습니다.

관광객 분들께는 수원 화성을 중심으로 주변 명소를 함께 둘러보시는 것을 추천드립니다!


In [17]:
# 대화 히스토리 출력
history = get_chat_history("tourist_1")

pprint(history.messages)

[HumanMessage(content='수원에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='수원은 역사와 문화가 풍부한 도시로, 다양한 볼거리와 즐길 거리가 많습니다. 수원에서 가볼 만한 주요 명소를 추천해드릴게요.\n\n1. **수원 화성**  \n   - 조선 정조 시대에 건설된 성곽으로, 유네스코 세계문화유산에 등재되어 있습니다.  \n   - 화성행궁, 장안문, 팔달문 등 다양한 건축물을 둘러볼 수 있고, 성곽을 따라 산책하기 좋습니다.  \n   - 야간에는 조명으로 아름답게 빛나는 화성도 감상할 수 있습니다.\n\n2. **화성행궁**  \n   - 조선시대 임금이 수원에 머물 때 사용하던 임시 궁궐입니다.  \n   - 전통 건축과 다양한 문화행사가 열려 역사 체험에 좋습니다.\n\n3. **팔달문 시장**  \n   - 전통시장으로 다양한 먹거리와 쇼핑을 즐길 수 있습니다.  \n   - 수원 갈비, 떡갈비 등 지역 특산 음식도 맛볼 수 있어요.\n\n4. **수원박물관**  \n   - 수원의 역사와 문화를 한눈에 볼 수 있는 곳입니다.  \n   - 가족 단위 방문객에게 추천합니다.\n\n5. **광교 호수공원**  \n   - 자연과 함께 산책하거나 자전거 타기 좋은 공원입니다.  \n   - 카페와 레스토랑도 많아 휴식 공간으로 적합합니다.\n\n6. **경기도박물관**  \n   - 경기도 지역의 역사와 문화를 소개하는 박물관입니다.  \n   - 다양한 전시와 체험 프로그램이 마련되어 있습니다.\n\n이 외에도 수원에는 다양한 카페, 맛집, 문화 공간이 많으니 일정에 맞게 방문해 보세요. 도움이 필요하시면 언제든지 말씀해 주세요!', additional_kwargs={'refusal': None}, response_metadata={}),
 HumanMessage(content='이전에 추천한 장소 중에서 가장 인기 있는 곳은 어

In [18]:
# 대화 히스토리 초기화
history.clear()

In [19]:
# 대화 히스토리 출력
pprint(history.messages)

[]


# **메시지 관리 기법**

* 메시지 트리밍은 컨텍스트 윈도우의 토큰 제한을 관리하며, 시스템 메시지 포함 여부와 시작 위치 등을 세밀하게 제어할 수 있습니다.

* 장시간 진행되는 대화의 경우, 이전 대화 내용을 한 문장으로 요약하여 컨텍스트로 활용함으로써 메모리 효율성을 높일 수 있습니다.

* 대화 히스토리가 일정 길이를 초과할 경우, 요약된 내용과 최근 메시지만을 새로운 히스토리로 구성하여 컨텍스트의 품질을 유지하면서도 토큰 사용량을 최적화할 수 있습니다.

### 1. **메시지 트리밍(Message Trimming)**

* `trim_messages` 함수는 컨텍스트 윈도우의 토큰 제한을 관리하기 위한 핵심 도구로, 시스템 메시지를 포함할지 여부와 어디서부터 트리밍을 시작할지 등을 상세하게 설정할 수 있습니다.

* 트리밍 전략으로 "last" 옵션을 사용하면 가장 최근의 메시지부터 시작하여 지정된 토큰 제한에 맞춰 이전 메시지들을 선택적으로 포함시킬 수 있습니다.

In [20]:
from langchain_core.messages import trim_messages

# 메시지 목록을 정의
orginal_messages = [
    HumanMessage(content="안녕하세요. 제 이름은 홍길동입니다."),
    AIMessage(content="안녕하세요! 어떻게 도와드릴까요?"),
    HumanMessage(content="제 이름을 기억하나요?")
]

# 트리머(trimmer) 생성
# 마지막 메시지 2개만 유지하도록 설정 (token은 메시지를 나타냄) 
trimmer = trim_messages(strategy="last", max_tokens=2, token_counter=len)

trimmed_messages = trimmer.invoke(orginal_messages)

pprint(trimmed_messages)

[AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='제 이름을 기억하나요?', additional_kwargs={}, response_metadata={})]


In [22]:
from langchain_core.messages.utils import count_tokens_approximately
trimmer = trim_messages(strategy="last", max_tokens=50, token_counter=count_tokens_approximately)
trimmed_messages = trimmer.invoke(orginal_messages)
pprint(trimmed_messages)

[HumanMessage(content='안녕하세요. 제 이름은 홍길동입니다.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='제 이름을 기억하나요?', additional_kwargs={}, response_metadata={})]


In [23]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from pydantic import BaseModel, Field
from typing import List

# 메시지 트리밍이 적용된 인메모리 히스토리 구현
class TrimmedInMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    max_tokens: int = Field(default=2)  # 유지할 최대 메시지 수
    
    def __init__(self, max_tokens: int = 2, **kwargs):
        """
        TrimmedInMemoryHistory 초기화
        
        Args:
            max_tokens (int): 유지할 최대 메시지 수
            **kwargs: 추가 키워드 인자
        """
        super().__init__(max_tokens=max_tokens, **kwargs)
    
    def add_messages(self, messages: List[BaseMessage]) -> None:
        self.messages.extend(messages)
        # 메시지 추가 후 트리밍 수행
        trimmer = trim_messages(
            strategy="last",
            max_tokens=self.max_tokens,
            token_counter=len
        )
        self.messages = trimmer.invoke(self.messages)
    
    def clear(self) -> None:
        self.messages = []

# 세션 저장소
store = {}

# 세션 ID로 트리밍된 히스토리 가져오기
def get_trimmed_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = TrimmedInMemoryHistory()
    return store[session_id]

# 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 여행 가이드입니다. 관광객에게 유용한 정보를 제공하세요."),
    MessagesPlaceholder(variable_name="history"), 
    ("human", "{input}")
])

# 프롬프트와 llm을 연결하여 체인 생성
chain = prompt | llm

# 트리밍된 히스토리 관리 추가
chain_with_trimmed_history = RunnableWithMessageHistory(
    chain,
    get_trimmed_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 지정된 세션 ID(tourist_1)를 사용하여 체인 실행
response = chain_with_trimmed_history.invoke(
    {
        "input": "서울에서 가볼만한 곳을 추천해주세요."
    },
    config={"configurable": {"session_id": "tourist_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

여행 가이드 답변:
서울에서 가볼 만한 곳을 추천해드릴게요!

1. 경복궁 – 조선 시대의 대표 궁궐로, 전통 건축과 아름다운 정원을 감상할 수 있어요. 한복을 입고 방문하면 더 특별한 경험이 됩니다.
2. 북촌 한옥마을 – 전통 한옥이 잘 보존된 마을로, 골목길을 걸으며 한국의 옛 생활 문화를 느낄 수 있습니다.
3. 명동 – 쇼핑과 맛집이 가득한 번화가로, 다양한 패션 브랜드와 길거리 음식을 즐길 수 있어요.
4. N서울타워 – 서울의 전경을 한눈에 볼 수 있는 전망대로, 특히 야경이 아름답습니다.
5. 홍대 – 젊음의 거리로 유명하며, 예술과 음악, 카페 문화가 활발한 곳입니다.
6. 동대문 디자인 플라자(DDP) – 현대적인 건축물과 다양한 전시, 야시장으로 유명한 장소입니다.
7. 한강공원 – 한강을 따라 조성된 공원에서 자전거 타기, 피크닉, 야경 감상 등 여유로운 시간을 보낼 수 있어요.

필요하시면 교통편이나 맛집 추천도 도와드릴게요!


In [24]:
# 대화 히스토리 출력
history = get_trimmed_session_history("tourist_1")

pprint(history.messages)

[HumanMessage(content='서울에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='서울에서 가볼 만한 곳을 추천해드릴게요!\n\n1. 경복궁 – 조선 시대의 대표 궁궐로, 전통 건축과 아름다운 정원을 감상할 수 있어요. 한복을 입고 방문하면 더 특별한 경험이 됩니다.\n2. 북촌 한옥마을 – 전통 한옥이 잘 보존된 마을로, 골목길을 걸으며 한국의 옛 생활 문화를 느낄 수 있습니다.\n3. 명동 – 쇼핑과 맛집이 가득한 번화가로, 다양한 패션 브랜드와 길거리 음식을 즐길 수 있어요.\n4. N서울타워 – 서울의 전경을 한눈에 볼 수 있는 전망대로, 특히 야경이 아름답습니다.\n5. 홍대 – 젊음의 거리로 유명하며, 예술과 음악, 카페 문화가 활발한 곳입니다.\n6. 동대문 디자인 플라자(DDP) – 현대적인 건축물과 다양한 전시, 야시장으로 유명한 장소입니다.\n7. 한강공원 – 한강을 따라 조성된 공원에서 자전거 타기, 피크닉, 야경 감상 등 여유로운 시간을 보낼 수 있어요.\n\n필요하시면 교통편이나 맛집 추천도 도와드릴게요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 288, 'prompt_tokens': 40, 'total_tokens': 328, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint

In [25]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_trimmed_history.invoke(
    {
        "input": "이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?"
    },
    config={"configurable": {"session_id": "tourist_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

여행 가이드 답변:
이전에 추천드린 장소 중에서 가장 인기 있는 곳은 **경복궁**과 **명동**, 그리고 **N서울타워**입니다.

- **경복궁**은 한국의 역사와 문화를 체험할 수 있는 대표적인 관광지로, 외국인 관광객뿐만 아니라 국내 방문객에게도 매우 인기가 많아요.
- **명동**은 쇼핑과 먹거리가 풍부해 특히 젊은 층과 관광객들에게 사랑받는 번화가입니다.
- **N서울타워**는 서울의 전경과 야경을 감상할 수 있는 명소로, 데이트 코스나 가족 나들이 장소로도 인기가 높습니다.

이 세 곳은 서울 방문 시 꼭 들러보는 필수 코스라고 할 수 있어요! 방문 목적에 따라 선택하시면 좋습니다.


In [26]:
# 대화 히스토리 출력
history = get_trimmed_session_history("tourist_1")

pprint(history.messages)

[HumanMessage(content='이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='이전에 추천드린 장소 중에서 가장 인기 있는 곳은 **경복궁**과 **명동**, 그리고 **N서울타워**입니다.\n\n- **경복궁**은 한국의 역사와 문화를 체험할 수 있는 대표적인 관광지로, 외국인 관광객뿐만 아니라 국내 방문객에게도 매우 인기가 많아요.\n- **명동**은 쇼핑과 먹거리가 풍부해 특히 젊은 층과 관광객들에게 사랑받는 번화가입니다.\n- **N서울타워**는 서울의 전경과 야경을 감상할 수 있는 명소로, 데이트 코스나 가족 나들이 장소로도 인기가 높습니다.\n\n이 세 곳은 서울 방문 시 꼭 들러보는 필수 코스라고 할 수 있어요! 방문 목적에 따라 선택하시면 좋습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 181, 'prompt_tokens': 352, 'total_tokens': 533, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_a150906e27', 'id': 'chatcmpl-CEvci6biHw0XcTZOJEkYlbpT032hM', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': Non

In [27]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_trimmed_history.invoke(
    {
        "input": "이 장소의 정식 명칭은 무엇인가요?"
    },
    config={"configurable": {"session_id": "tourist_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

여행 가이드 답변:
말씀드린 장소들의 정식 명칭은 다음과 같습니다:

- 경복궁: **경복궁 (Gyeongbokgung Palace)**
- 명동: **명동 (Myeong-dong Shopping District)**
- N서울타워: **N서울타워 (N Seoul Tower)** 또는 **남산서울타워 (Namsan Seoul Tower)**

각 장소는 공식 명칭과 함께 영어 이름도 널리 사용되고 있으니, 여행 중 안내판이나 지도에서 쉽게 찾으실 수 있습니다.


In [28]:
# 대화 히스토리 출력
history = get_trimmed_session_history("tourist_1")

pprint(history.messages)

[HumanMessage(content='이 장소의 정식 명칭은 무엇인가요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='말씀드린 장소들의 정식 명칭은 다음과 같습니다:\n\n- 경복궁: **경복궁 (Gyeongbokgung Palace)**\n- 명동: **명동 (Myeong-dong Shopping District)**\n- N서울타워: **N서울타워 (N Seoul Tower)** 또는 **남산서울타워 (Namsan Seoul Tower)**\n\n각 장소는 공식 명칭과 함께 영어 이름도 널리 사용되고 있으니, 여행 중 안내판이나 지도에서 쉽게 찾으실 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 114, 'prompt_tokens': 246, 'total_tokens': 360, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4fce0778af', 'id': 'chatcmpl-CEvclOmMq3i85ZcWRRI74NTZy4YQL', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--086b41f0-3c0f-4c13-9d39-2ba6ba9695da-0', usage_metadata={'input_tokens': 246, 

### 2. **대화 요약 저장**

* 대화가 길어질 경우, 전체 대화 내용을 요약하여 컨텍스트로 활용하는 방식으로 이전 대화의 핵심을 추출합니다.

* 일반적으로 메시지 히스토리가 지정된 길이(예: 4개의 메시지)를 초과할 경우, 이전 대화들을 요약하고 가장 최근의 메시지만 유지하는 방식으로 새로운 대화 히스토리를 구성합니다.

* 이러한 요약 메모리 방식을 통해 토큰 사용량을 크게 줄이면서도 대화의 핵심 문맥을 유지할 수 있으며, 특히 장시간 진행되는 대화에서 효과적입니다.

In [29]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from typing import List
    
class SummarizedInMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    summary_threshold: int = Field(default=6)  # 요약을 시작할 메시지 수
    # 요약에 사용할 LLM - lambda를 사용하여 필드가 초기화될 때마다 새로운 ChatOpenAI 인스턴스 생성
    llm: ChatOpenAI = Field(default_factory=lambda: ChatOpenAI(model="gpt-4.1-mini", temperature=0.1, top_p=0.9))  
    
    def add_messages(self, new_messages: List[BaseMessage]) -> None:
        self.messages.extend(new_messages)

        print(f"메시지 수: {len(self.messages)}")
        
        # 메시지 수가 임계값을 넘으면 요약 수행
        if len(self.messages) >= self.summary_threshold:
            # 마지막 사용자 메시지 저장 (HumanMessage, AIMessage 순서로 생성)
            last_user_message = self.messages[-2]
            last_ai_message = self.messages[-1]
            
            # 요약 생성
            summary_prompt = (
                "Distill the above chat messages into a single summary message. "
                "Include as many specific details as you can."
                "Use the original language and tone of the conversation."
            )
            
            summary_chain_messages = [
                SystemMessage(content=(
                    "You are a helpful assistant. "
                    "Your task is to summarize the conversation accurately."
                )),
                *self.messages[:-2],  # 마지막 대화 턴을 제외한 모든 메시지
                HumanMessage(content=summary_prompt)
            ]
            
            # 요약 생성
            summary = self.llm.invoke(summary_chain_messages)
            
            # 메시지 리스트 초기화 후 요약과 마지막 메시지 추가
            self.messages = [
                summary,
                last_user_message,
                last_ai_message
            ]
    
    def clear(self) -> None:
        self.messages = []

# 세션 저장소
store = {}

# 세션 ID로 요약된 히스토리 가져오기
def get_summarized_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = SummarizedInMemoryHistory()
    return store[session_id]


# 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 여행 가이드입니다. 관광객에게 유용한 정보를 제공하세요."),
    MessagesPlaceholder(variable_name="history"), 
    ("human", "{input}")
])

# 프롬프트와 llm을 연결하여 체인 생성
chain = prompt | llm

# 체인 구성
chain_with_summarized_history = RunnableWithMessageHistory(
    chain,
    get_summarized_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 지정된 세션 ID(user_1)를 사용하여 체인 실행
response = chain_with_summarized_history.invoke(
    {
        "input": "서울에서 가볼만한 곳을 추천해주세요."
    },
    config={"configurable": {"session_id": "user_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

메시지 수: 2
여행 가이드 답변:
서울에는 볼거리와 즐길 거리가 많아 여행객들에게 인기 있는 도시입니다. 몇 가지 추천 장소를 소개해드릴게요.

1. 경복궁: 조선 시대의 대표 궁궐로, 전통 건축과 아름다운 정원을 감상할 수 있습니다. 한복을 입고 방문하면 더욱 특별한 경험이 됩니다.

2. 북촌 한옥마을: 전통 한옥이 잘 보존된 마을로, 골목길을 걸으며 한국의 옛 생활 문화를 느낄 수 있습니다.

3. 명동: 쇼핑과 먹거리가 풍부한 번화가로, 한국 패션과 길거리 음식을 즐기기에 좋습니다.

4. N서울타워: 남산 정상에 위치한 전망대로, 서울 전경을 한눈에 볼 수 있습니다. 야경이 특히 아름답습니다.

5. 홍대: 젊음의 거리로 유명하며, 다양한 카페, 공연장, 예술 공간이 많아 활기찬 분위기를 즐길 수 있습니다.

6. 동대문 디자인 플라자(DDP): 현대적인 건축물과 함께 패션, 디자인 전시가 열리는 곳으로, 야간 조명이 인상적입니다.

이 외에도 한강공원에서 자전거 타기, 인사동에서 전통 공예품 구경 등 다양한 체험이 가능합니다. 여행 일정과 취향에 맞게 방문해 보세요!


In [30]:
# 대화 히스토리 출력
history = get_summarized_session_history("user_1")

pprint(history.messages)

[HumanMessage(content='서울에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='서울에는 볼거리와 즐길 거리가 많아 여행객들에게 인기 있는 도시입니다. 몇 가지 추천 장소를 소개해드릴게요.\n\n1. 경복궁: 조선 시대의 대표 궁궐로, 전통 건축과 아름다운 정원을 감상할 수 있습니다. 한복을 입고 방문하면 더욱 특별한 경험이 됩니다.\n\n2. 북촌 한옥마을: 전통 한옥이 잘 보존된 마을로, 골목길을 걸으며 한국의 옛 생활 문화를 느낄 수 있습니다.\n\n3. 명동: 쇼핑과 먹거리가 풍부한 번화가로, 한국 패션과 길거리 음식을 즐기기에 좋습니다.\n\n4. N서울타워: 남산 정상에 위치한 전망대로, 서울 전경을 한눈에 볼 수 있습니다. 야경이 특히 아름답습니다.\n\n5. 홍대: 젊음의 거리로 유명하며, 다양한 카페, 공연장, 예술 공간이 많아 활기찬 분위기를 즐길 수 있습니다.\n\n6. 동대문 디자인 플라자(DDP): 현대적인 건축물과 함께 패션, 디자인 전시가 열리는 곳으로, 야간 조명이 인상적입니다.\n\n이 외에도 한강공원에서 자전거 타기, 인사동에서 전통 공예품 구경 등 다양한 체험이 가능합니다. 여행 일정과 취향에 맞게 방문해 보세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 309, 'prompt_tokens': 40, 'total_tokens': 349, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_token

In [31]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_summarized_history.invoke(
    {
        "input": "이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?"
    },
    config={"configurable": {"session_id": "user_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

메시지 수: 4
여행 가이드 답변:
이전에 추천한 장소 중에서 가장 인기 있는 곳은 경복궁과 N서울타워입니다.

- 경복궁은 한국의 대표적인 역사 유적지로, 한국 전통 문화와 건축을 체험할 수 있어 외국인 관광객과 내국인 모두에게 매우 인기가 많습니다.

- N서울타워는 서울의 랜드마크로, 특히 야경을 감상하기 좋은 장소로 연인이나 가족 단위 방문객들에게 사랑받고 있습니다.

두 곳 모두 서울을 방문하는 관광객이라면 꼭 들러볼 만한 명소로 꼽힙니다. 방문 목적에 따라 역사와 전통을 느끼고 싶다면 경복궁, 서울 전경과 야경을 즐기고 싶다면 N서울타워를 추천드립니다.


In [32]:
# 대화 히스토리 출력
history = get_summarized_session_history("user_1")

pprint(history.messages)

[HumanMessage(content='서울에서 가볼만한 곳을 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='서울에는 볼거리와 즐길 거리가 많아 여행객들에게 인기 있는 도시입니다. 몇 가지 추천 장소를 소개해드릴게요.\n\n1. 경복궁: 조선 시대의 대표 궁궐로, 전통 건축과 아름다운 정원을 감상할 수 있습니다. 한복을 입고 방문하면 더욱 특별한 경험이 됩니다.\n\n2. 북촌 한옥마을: 전통 한옥이 잘 보존된 마을로, 골목길을 걸으며 한국의 옛 생활 문화를 느낄 수 있습니다.\n\n3. 명동: 쇼핑과 먹거리가 풍부한 번화가로, 한국 패션과 길거리 음식을 즐기기에 좋습니다.\n\n4. N서울타워: 남산 정상에 위치한 전망대로, 서울 전경을 한눈에 볼 수 있습니다. 야경이 특히 아름답습니다.\n\n5. 홍대: 젊음의 거리로 유명하며, 다양한 카페, 공연장, 예술 공간이 많아 활기찬 분위기를 즐길 수 있습니다.\n\n6. 동대문 디자인 플라자(DDP): 현대적인 건축물과 함께 패션, 디자인 전시가 열리는 곳으로, 야간 조명이 인상적입니다.\n\n이 외에도 한강공원에서 자전거 타기, 인사동에서 전통 공예품 구경 등 다양한 체험이 가능합니다. 여행 일정과 취향에 맞게 방문해 보세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 309, 'prompt_tokens': 40, 'total_tokens': 349, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_token

In [33]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_summarized_history.invoke(
    {
        "input": "전주에서 가볼만한 곳을 추천해주세요."
    },
    config={"configurable": {"session_id": "user_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

메시지 수: 6
여행 가이드 답변:
전주는 전통과 맛이 어우러진 도시로, 한국의 전통 문화를 체험하기에 좋은 곳입니다. 전주에서 가볼 만한 주요 명소를 소개해드릴게요.

1. 전주 한옥마을: 전통 한옥이 잘 보존된 마을로, 한복 체험, 전통 공예품 구경, 전통 음식 맛보기 등 다양한 체험이 가능합니다. 특히 전주비빔밥과 한과 같은 지역 음식도 꼭 맛보세요.

2. 경기전: 조선 태조 이성계의 어진(초상화)이 모셔진 곳으로, 역사와 전통을 느낄 수 있는 중요한 문화재입니다.

3. 전동성당: 한국 최초의 고딕 양식 성당으로, 아름다운 건축물과 평화로운 분위기를 즐길 수 있습니다.

4. 오목대와 이목대: 전주를 한눈에 내려다볼 수 있는 전망대로, 자연 속에서 산책하기 좋습니다.

5. 전주 남부시장: 다양한 먹거리와 전통 시장의 활기를 느낄 수 있는 곳으로, 전주만의 특색 있는 음식들을 맛볼 수 있습니다.

6. 덕진공원: 호수와 정원이 아름다운 공원으로, 산책이나 휴식을 취하기에 좋습니다.

전주는 전통문화와 맛집이 풍부한 도시이니, 여유롭게 한옥마을을 중심으로 다양한 체험과 음식을 즐겨보시길 추천드립니다!


In [34]:
# 대화 히스토리 출력
history = get_summarized_session_history("user_1")

pprint(history.messages)

[AIMessage(content='서울에서 가볼 만한 인기 장소로는 경복궁과 N서울타워가 있습니다. 경복궁은 조선 시대 대표 궁궐로 전통 건축과 정원을 감상할 수 있으며, 한복 체험도 가능합니다. N서울타워는 남산 정상에 위치해 서울 전경과 특히 아름다운 야경을 즐길 수 있는 랜드마크입니다. 두 곳 모두 외국인과 내국인 관광객에게 매우 사랑받는 명소로, 역사와 전통을 느끼고 싶다면 경복궁, 서울의 전경과 야경을 보고 싶다면 N서울타워 방문을 추천드립니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 140, 'prompt_tokens': 577, 'total_tokens': 717, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6d7dcc9a98', 'id': 'chatcmpl-CEvd5WNOh9sT71VYUJ5U7LF2vovPL', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--6c10fc9b-2e80-4ae7-9857-ea35f93bd54e-0', usage_metadata={'input_tokens': 577, 'output_tokens': 140, 'total_tokens': 717, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_to

In [ ]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_summarized_history.invoke(
    {
        "input": "두 도시에서 서로 비슷한 장소가 어디인가요?"
    },
    config={"configurable": {"session_id": "user_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

메시지 수: 5
여행 가이드 답변:
서울과 전주에서 비슷한 성격의 장소를 비교해보면 다음과 같습니다:

1. 전통 한옥마을  
- 서울: 북촌 한옥마을 — 전통 한옥이 모여 있는 마을로, 한국 전통문화 체험과 산책에 좋습니다.  
- 전주: 전주한옥마을 — 규모가 더 크고 전통 한옥이 잘 보존되어 있으며, 한복 체험과 전통 음식도 즐길 수 있습니다.

2. 역사적 궁궐/문화재  
- 서울: 경복궁 — 조선시대 대표 궁궐로, 한국 전통 건축과 역사를 체험할 수 있는 곳입니다.  
- 전주: 경기전 — 조선 태조 이성계의 어진을 모신 곳으로, 역사적 의미가 깊은 문화재입니다.

3. 전통 시장  
- 서울: 남대문시장, 광장시장 등 — 다양한 먹거리와 쇼핑을 즐길 수 있는 전통 시장입니다.  
- 전주: 남부시장 — 전주 특산물과 길거리 음식을 즐길 수 있는 재래시장입니다.

4. 종교 건축물  
- 서울: 조계사, 명동성당 등 — 역사적이고 아름다운 사찰과 성당이 있습니다.  
- 전주: 전동성당 — 고딕 양식의 아름다운 성당으로 전통과 현대가 어우러진 장소입니다.

두 도시 모두 전통과 현대가 조화를 이루는 관광지가 많아, 각각의 특색을 느끼면서 비슷한 유형의 장소를 즐기실 수 있습니다.


In [ ]:
# 대화 히스토리 출력
history = get_summarized_session_history("user_1")

pprint(history.messages)

[AIMessage(content='서울에서 가볼 만한 곳으로 경복궁, 북촌 한옥마을, 명동, N서울타워, 동대문 디자인 플라자(DDP), 한강공원을 추천드렸습니다. 그중 가장 인기 있는 곳은 경복궁과 N서울타워인데요, 경복궁은 한국 전통문화 체험과 사진 명소로 내외국인 모두에게 사랑받고, N서울타워는 서울 야경 명소이자 연인들의 데이트 코스로 유명해 꾸준히 방문객이 많습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 120, 'prompt_tokens': 556, 'total_tokens': 676, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6d7dcc9a98', 'id': 'chatcmpl-CEYliYYfKbEwJGHkVaXsINKR8AtrJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d10b4869-6677-4bbc-8c7f-9454ffd6fa35-0', usage_metadata={'input_tokens': 556, 'output_tokens': 120, 'total_tokens': 676, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),

In [ ]:
# 이전 대화 내용을 기반으로 새로운 질문을 추가하여 체인 실행
response = chain_with_summarized_history.invoke(
    {
        "input": "두 도시에서 각각 한 장소만 추천해주세요."
    },
    config={"configurable": {"session_id": "user_1"}}
)

print(f"여행 가이드 답변:\n{response.content}")

메시지 수: 7
여행 가이드 답변:
서울에서는 경복궁을 추천드립니다. 조선시대 대표 궁궐로서 한국 전통 건축과 역사를 깊이 체험할 수 있고, 광화문 광장과도 가까워 주변 관광도 편리합니다.

전주에서는 전주한옥마을을 추천드립니다. 전통 한옥이 잘 보존된 마을로, 한복 체험과 전통 음식, 다양한 문화 행사를 즐길 수 있어 전주의 매력을 한눈에 느낄 수 있는 명소입니다.


In [ ]:
# 대화 히스토리 출력
history = get_summarized_session_history("user_1")

pprint(history.messages)

[AIMessage(content='서울에서 가볼 만한 곳으로 경복궁, 북촌 한옥마을, 명동, N서울타워, 동대문 디자인 플라자(DDP), 한강공원을 추천드렸고, 특히 경복궁과 N서울타워가 인기 명소입니다. 전주에서는 전주한옥마을, 경기전, 전동성당, 남부시장, 덕진공원, 전주향교를 추천하며, 전주한옥마을은 한복 체험과 전통 음식으로 유명합니다. 서울과 전주에서 비슷한 장소로는 전통 한옥마을(서울 북촌 한옥마을 vs. 전주한옥마을), 역사적 궁궐/문화재(서울 경복궁 vs. 전주 경기전), 전통 시장(서울 남대문시장 등 vs. 전주 남부시장), 종교 건축물(서울 명동성당 등 vs. 전주 전동성당)이 있어 두 도시 모두 전통과 현대가 조화를 이루는 관광지를 즐길 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 231, 'prompt_tokens': 881, 'total_tokens': 1112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_a150906e27', 'id': 'chatcmpl-CEYlqM0wLRq7dbvVaxwWNUL5SqdOI', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8e919b0d-96ab-477e-9c32-60dd01638140-0', usage_metadata={'inp

In [ ]:
# 대화 히스토리 초기화
history.clear()

In [ ]:
# 대화 히스토리 출력
history = get_summarized_session_history("user_1")

pprint(history.messages)

[]


---
# **[실습]**

- 메시지 트리밍과 대화 요약 저장을 결합하여 메시지를 관리하는 기능을 구현합니다. 

1. 새로운 클래스 구조

In [38]:
class TrimmedAndSummarizedHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage]
    max_tokens: int  # 트리밍 기준
    summary_threshold: int  # 요약 기준
    llm: ChatOpenAI
    summarized_messages: List[BaseMessage]  # 요약된 메시지 저장용

2. 메시지 처리 로직
- 새 메시지 추가시 max_tokens 체크
- 트리밍 발생하면 제거될 메시지 식별
- 제거 예정 메시지들은 요약하여 summarized_messages에 저장
- 현재 메시지는 트리밍된 상태로 유지

3. 요약 프로세스
- 트리밍으로 제거될 메시지들만 선별
- 선별된 메시지들에 대해 summary_chain 실행
- 요약본을 시스템 메시지로 변환하여 저장

In [41]:
class SummarizingAndTrimmingHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    summarized_messages: List[BaseMessage] = Field(default_factory=list) # 요약본 저장 리스트
    max_tokens: int = 4  # 이 개수를 초과하면 요약 및 트리밍 실행 (데모를 위해 작게 설정)
    llm: ChatOpenAI = Field(default_factory=lambda: ChatOpenAI(model="gpt-4o-mini", temperature=0.1))

    def add_messages(self, new_messages: List[BaseMessage]) -> None:
        """새 메시지를 추가하고, max_tokens를 초과하면 요약 및 트리밍을 수행합니다."""
        self.messages.extend(new_messages)

        # 메시지 수가 max_tokens를 초과하면 요약 및 트리밍 프로세스 시작
        if len(self.messages) > self.max_tokens:
            print(f"========= 임계값 도달 ({len(self.messages)} > {self.max_tokens}). 요약을 시작합니다. =========")

            # 1. 요약할 메시지(오래된 메시지)와 유지할 메시지(최신 메시지) 분리
            # 예: max_tokens=4이고 메시지가 6개면, 앞 2개(6-4)를 요약하고 뒤 4개를 유지
            num_messages_to_summarize = len(self.messages) - self.max_tokens
            messages_to_summarize = self.messages[:num_messages_to_summarize]
            messages_to_keep = self.messages[num_messages_to_summarize:]

            # 2. 요약 프롬프트 실행
            summary_prompt = (
                "다음 대화 내용을 하나의 요약 메시지로 압축해주세요. "
                "대화의 핵심적인 맥락과 주요 정보를 포함해야 합니다."
            )
            summary_chain_messages = [
                SystemMessage(content="You are a helpful assistant tasked with summarizing conversations."),
                *messages_to_summarize,
                HumanMessage(content=summary_prompt)
            ]
            
            summary_response = self.llm.invoke(summary_chain_messages)
            
            # 3. 요약본을 시스템 메시지로 변환하여 저장
            summary_message = SystemMessage(content=f"지난 대화 요약: {summary_response.content}")
            self.summarized_messages.append(summary_message)
            
            # 4. 현재 메시지를 "요약본 + 유지할 최신 메시지"로 재구성
            self.messages = self.summarized_messages + messages_to_keep

    def clear(self) -> None:
        self.messages = []
        self.summarized_messages = []

# --- 체인 실행 부분 ---

# 세션별 대화 기록을 저장할 저장소
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """세션 ID에 해당하는 대화 기록 인스턴스를 가져옵니다."""
    if session_id not in store:
        # 새로 정의한 History 클래스를 사용
        store[session_id] = SummarizingAndTrimmingHistory()
    return store[session_id]

# 프롬프트 템플릿
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 지식이 풍부한 여행 전문가입니다."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])

# LangChain 실행 체인 구성
chain = prompt | llm

# 대화 기록 관리 기능을 체인에 연결
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

# --- 대화 시뮬레이션 ---
session_id = "tourist_A"
questions = [
    "안녕하세요! 서울 여행 계획 중인데, 첫째 날 가볼 만한 곳 추천해주세요.",
    "좋은 추천이네요! 그럼 경복궁 근처에 맛있는 한정식 식당도 알려주실 수 있나요?",
    "서울 근처에 있는 전통 찻집도 추천해 주세요.",
    "서울에서 경험 할 수 있는 이색 활동이 있을까요?",
    "감사합니다. 마지막으로, 저녁에 남산타워에 가려고 하는데 가는 방법 좀 알려주세요."
]

for i, question in enumerate(questions):
    print(f"\n--- 대화 {i+1} ---")
    print(f"질문: {question}")
    
    response = chain_with_history.invoke(
        {"input": question},
        config={"configurable": {"session_id": session_id}}
    )
    
    print(f"답변: {response.content}")
    print("\n--- 현재 대화 기록 상태 ---")
    print(store[session_id].messages)


--- 대화 1 ---
질문: 안녕하세요! 서울 여행 계획 중인데, 첫째 날 가볼 만한 곳 추천해주세요.
답변: 안녕하세요! 서울 첫째 날 여행 코스로 다음을 추천드려요:

1. 경복궁 – 조선 시대 대표 궁궐로, 한복 체험도 가능해요.
2. 북촌 한옥마을 – 전통 한옥이 모여 있는 예쁜 골목길 산책하기 좋아요.
3. 인사동 – 전통 공예품과 갤러리, 카페가 많아 한국 문화를 느끼기 좋아요.
4. 청계천 – 도심 속 하천 산책로로 저녁에 조명이 아름답습니다.
5. 명동 – 쇼핑과 길거리 음식 즐기기에 최적의 장소예요.

이 코스는 서울의 전통과 현대를 함께 경험할 수 있어 첫날 일정으로 적합합니다. 필요하면 교통편이나 식당 추천도 도와드릴게요!

--- 현재 대화 기록 상태 ---
[HumanMessage(content='안녕하세요! 서울 여행 계획 중인데, 첫째 날 가볼 만한 곳 추천해주세요.', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 서울 첫째 날 여행 코스로 다음을 추천드려요:\n\n1. 경복궁 – 조선 시대 대표 궁궐로, 한복 체험도 가능해요.\n2. 북촌 한옥마을 – 전통 한옥이 모여 있는 예쁜 골목길 산책하기 좋아요.\n3. 인사동 – 전통 공예품과 갤러리, 카페가 많아 한국 문화를 느끼기 좋아요.\n4. 청계천 – 도심 속 하천 산책로로 저녁에 조명이 아름답습니다.\n5. 명동 – 쇼핑과 길거리 음식 즐기기에 최적의 장소예요.\n\n이 코스는 서울의 전통과 현대를 함께 경험할 수 있어 첫날 일정으로 적합합니다. 필요하면 교통편이나 식당 추천도 도와드릴게요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 187, 'prompt_tokens': 44, 'total_tokens': 231, 'completion_tokens_details': {